In [0]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import time
import warnings
import pickle as pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import gc

import zipfile

tqdm.pandas()

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [0]:
!unzip "/content/drive/My Drive/recordings.zip"

In [0]:
SAMPLE_RATE = 22050
warnings.filterwarnings('ignore')
df = pd.read_csv("drive/My Drive/aug_speakers.csv")
df.drop(df.columns[9:12],axis = 1, inplace = True)
print(df.columns)
df.describe()

In [0]:
import collections
samples_count = collections.Counter(df["native_language"].values)
new_d = {}
for key,val in samples_count.items():
    new_d[val] = new_d.get(val,[])+[key]

for i in range(8):
    if i in new_d.keys():
        del new_d[i]

# print(new_d)
sorted_d = {k: v for k, v in sorted(new_d.items(), key=lambda item: item[0])}
classes = []
for _,val in sorted_d.items():
    classes+=val
sorted_d.clear()
new_d.clear()

# print(total)
# print(len(classes))
# {k: v for k, v in sorted(new_d.items(), key=lambda item: item[0])}

In [0]:
import csv
def run_function(func,data,sampling_rate,age,sex):
    if func == "pitch":
        if sex == "male":
            y = librosa.effects.pitch_shift(data, sampling_rate, n_steps=4)
        else:
            y = librosa.effects.pitch_shift(data, sampling_rate, n_steps=-4)
    elif func == "faster":
        y = librosa.effects.time_stretch(data, 2.0)
    elif func == "slower":
        y = librosa.effects.time_stretch(data, 0.5)
    return y


def trim_pad(data):
    sr_dur = 22050*18
    if 0 < len(data): 
        y, _ = librosa.effects.trim(data) 
        trim_long_data = True
        if len(y) > sr_dur: 
            if trim_long_data:
                y = y[0:0+sr_dur]
        else: 
            padding = sr_dur - len(y)    
            offset = padding // 2
            y = np.pad(y, (offset, sr_dur - len(y) - offset), 'constant')
        return y
    return data
     

def append_new_row_to_df(new_row_data,file_counter):   
#     print("in append_new_data_to_df")
#     print(new_row_data)
    with open('drive/My Drive/aug_speakers.csv','a',newline='') as f:
        writer=csv.writer(f)
        new_row_data[3] = new_row_data[4]+str(file_counter)
        writer.writerow(new_row_data)
#     print("out append_new_data_to_df")
    

func_list = ["pitch","faster","slower"]
except_flag = False
for j,label in enumerate(classes):
    print(j+1,label)
    lang_df = df.loc[df['native_language'] == label]
    row_datas = []
    for _,row in lang_df.iterrows():
        if not row['file_missing?']:
            try:
                path = 'recordings/' + row['filename'] +".mp3"
                data, sampling_rate = librosa.load(path, sr=SAMPLE_RATE, duration=18)
                row_datas.append([data,sampling_rate,row.values])
            except Exception:
                print("exception at filename "+row['filename']) 

    row_datas_exists = len(row_datas)
    if row_datas_exists: #Means there was no exception when loading the file
        for func in func_list:
            i = 0
            new_row_datas = []
            while i < len(row_datas) and (len(row_datas)+i) < 50:
                new_data = run_function(func,row_datas[i][0],row_datas[i][1],row_datas[i][2][0],row_datas[i][2][5])
                new_row_datas.append([new_data,row_datas[i][1],row_datas[i][2].copy()])
                append_new_row_to_df(row_datas[i][2].copy(),len(row_datas)+i+1) #define this function
                i+=1
            row_datas += new_row_datas
        row_datas = row_datas[:50]
        print(len(row_datas))
        pickle_counter = 0
        for row_data in row_datas:
            trim_padded_y = trim_pad(row_data[0])
            mfcc = librosa.feature.mfcc(y=trim_padded_y, sr=sampling_rate, n_mfcc = 40)
            melspec = librosa.feature.melspectrogram(trim_padded_y, sr=sampling_rate, n_mels=128)
            save_data={"data": trim_padded_y,"sampling_rate":sampling_rate,"mfcc": mfcc,"melspec":melspec}
            pickle_counter+=1
            with open('Augmented_Features/'+row['native_language']+str(pickle_counter)+'.pkl', 'wb') as outfile:
                pickle.dump(save_data, outfile, pickle.HIGHEST_PROTOCOL)